In [88]:
import json
import numpy as np
import os

In [89]:
f_identities = open(os.path.join('raw_data', 'identity_CelebA.txt'), 'r')
identities = f_identities.read().split('\n')

f_attributes = open(os.path.join('raw_data', 'list_attr_celeba.txt'), 'r')
attributes = f_attributes.read().split('\n')

In [90]:
tar = 'Smiling'
sen_attr = 'Male'

target_idx = attributes[1].split().index(tar)
sen_idx = attributes[1].split().index(sen_attr)
image, target, sensitive = {},[],[]
#https://zhuanlan.zhihu.com/p/35975956

In [91]:
for line in attributes[2:]:
    info = line.split()
    if len(info) == 0:
        continue
    image_id = info[0]
    tar_img = (int(info[1:][target_idx]) + 1) / 2
    sen_img = (int(info[1:][sen_idx]) + 1) / 2

    image[image_id] = tar_img, sen_img
    target.append(tar_img)
    sensitive.append(sen_img)

In [92]:
import numpy as np
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader
from torchvision.transforms import transforms
from pathlib import Path
from PIL import Image
 
import cv2

In [96]:
import random
images_path = Path('raw_data//img_align_celeba')

images_list = list(images_path.glob('*.jpg')) # list(images_path.glob('*.png'))
images_list_str = [ str(x) for x in images_list ]
num = 20000
ids = random.sample(images_list_str, 20000)

sample_target = []
sample_sensitive = []
for path in ids:
    sample_target.append(image[path[-10:]][0])
    sample_sensitive.append(image[path[-10:]][1])

len(sample_sensitive)

20000

In [97]:
transform = transforms.Compose([
            transforms.ToTensor()      # 这里仅以最基本的为例
        ])
transform = transforms.Compose([
            transforms.CenterCrop((178, 178)), # 从图片中间切出224*224的图片
            transforms.Resize((128, 128)), # 缩放图片(Image)，保持长宽比不变，最短边为224像素
            transforms.ToTensor(), # 将图片(Image)转成Tensor，归一化至[0, 1]（直接除以255）
            # transforms.Normalize(mean=[.5, .5, .5], std=[.5, .5, .5]) # 标准化至[-1, 1]，规定均值和标准差
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # 常用标准化
        ])

In [98]:
X = torch.stack([transform(Image.open(image_path)) for image_path in ids[:20000]])
X.shape

torch.Size([20000, 3, 128, 128])

In [99]:
Y, A = torch.tensor(sample_target).reshape(-1,1), torch.tensor(sample_sensitive).reshape(-1,1)

In [103]:
a = np.array([1,2,3])
a[-1]

3

In [104]:
a = np.ones((3,4))

In [105]:
a[[1,2]]

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.]])

In [107]:
a[2:]

array([[1., 1., 1., 1.]])

In [ ]:
 with open(path, 'rb') as file:
            data_split = json.load(file)
        for client in data_split['users']:
            if name == 'celeba':
                X = split_celeba_data(data_split['user_data'][client]["x"]).astype(np.float32)
            else:
                X = np.array(data_split['user_data'][client]["x"]).astype(np.float32)

            Y = np.array(data_split['user_data'][client]["y"]).astype(np.float32).reshape(-1,1)

            A = np.array(data_split['user_data'][client]["A"]).astype(np.float32).reshape(-1,1)
                
            dataset = Fair_Dataset(X, Y, A)
            data_split['user_data'][client] = dataset